In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriasis"
cohort = "GSE226244"

# Input paths
in_trait_dir = "../../input/GEO/Psoriasis"
in_cohort_dir = "../../input/GEO/Psoriasis/GSE226244"

# Output paths
out_data_file = "../../output/preprocess/Psoriasis/GSE226244.csv"
out_gene_data_file = "../../output/preprocess/Psoriasis/gene_data/GSE226244.csv"
out_clinical_data_file = "../../output/preprocess/Psoriasis/clinical_data/GSE226244.csv"
json_path = "../../output/preprocess/Psoriasis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Total skin transcriptome modification induced by IL-17A blockade"
!Series_summary	"We studied psoriasis skin transcriptome modification induced by systemic IL-17A blockade with microarray analyses of total skin as part of a randomized placebo-controlled clinical trial (ClinicalTrial.gov identifier: NCT03131570)"
!Series_overall_design	"Whole tissue samples of (1) 33 psoriasis skin lesions before IL-17A blockade, (2) 28 psoriasis skin lesions after IL-17A blockade, and (3) 8 normal skin (including GSE78097 data) were obtained via skin biopsy and subjected to microarray analysis."
Sample Characteristics Dictionary:
{0: ['disease state: Psoriasis', 'disease state: Control'], 1: ['tissue: skin'], 2: ['lesional (ls) vs. normal: LS', 'lesional (ls) vs. normal: Normal'], 3: ['treatment: Pretreatment', 'treatment: Posttreatment'], 4: ['treatment_timeline: Baseline', 'treatment_timeline: on12', 'treatment_timeline: on24', 'treatment_timeline: off20']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray analysis data of skin samples
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Psoriasis):
# From the sample characteristics dictionary, key 0 contains 'disease state: Psoriasis' and 'disease state: Control'
# This indicates if the subject has Psoriasis or not
trait_row = 0

# For age:
# There is no age information in the sample characteristics dictionary
age_row = None

# For gender:
# There is no gender information in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary format (1 for Psoriasis, 0 for Control)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'psoriasis':
        return 1
    elif value.lower() == 'control':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous format"""
    # This function is defined but not used since age data is not available
    if pd.isna(value):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary format (0 for female, 1 for male)"""
    # This function is defined but not used since gender data is not available
    if pd.isna(value):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() in ['female', 'f']:
        return 0
    elif value.lower() in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # Sample Characteristics Dictionary from previous step:
    sample_chars = {
        0: ['disease state: Psoriasis', 'disease state: Control'],
        1: ['tissue: skin'],
        2: ['lesional (ls) vs. normal: LS', 'lesional (ls) vs. normal: Normal'],
        3: ['treatment: Pretreatment', 'treatment: Posttreatment'],
        4: ['treatment_timeline: Baseline', 'treatment_timeline: on12', 'treatment_timeline: on24', 'treatment_timeline: off20']
    }
    
    # Convert the sample characteristics to a DataFrame format that geo_select_clinical_features can process
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [1.0], 1: [0.0], 2: [nan], 3: [nan]}
Clinical data saved to ../../output/preprocess/Psoriasis/clinical_data/GSE226244.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene data dimensions: 54675 genes × 69 samples


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Affymetrix probe IDs, not human gene symbols
# They need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 3. Preview the gene annotation dataframe
print("Gene annotation dataframe column names:")
print(gene_annotation.columns)

# Preview the first few rows to understand the data structure
print("\nPreview of gene annotation data:")
annotation_preview = preview_df(gene_annotation, n=3)
print(annotation_preview)

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Gene annotation dataframe column names:
Index(['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date',
       'Sequence Type', 'Sequence Source', 'Target Description',
       'Representative Public ID', 'Gene Title', 'Gene Symbol',
       'ENTREZ_GENE_ID', 'RefSeq Transcript ID',
       'Gene Ontology Biological Process', 'Gene Ontology Cellular Component',
       'Gene Ontology Molecular Function'],
      dtype='object')

Preview of gene annotation data:
{'ID': ['1007_s_at', '1053_at', '117_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757'], 'SPOT_ID': [nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor ty

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which columns in gene annotation correspond to identifiers and gene symbols
# From the preview, we can see that 'ID' in gene_annotation matches the probe IDs in gene_data
# And 'Gene Symbol' contains the human gene symbols we need
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get gene mapping dataframe by extracting the two columns from gene annotation
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Print first few rows of the mapping dataframe
print("\nPreview of gene mapping DataFrame:")
print(mapping_df.head())
print(f"Shape of mapping dataframe: {mapping_df.shape}")

# 3. Apply gene mapping to convert probe-level measurements to gene-level expressions
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the resulting gene expression dataframe
print("\nAfter mapping to gene symbols:")
print(f"Gene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First 10 gene symbols:")
print(gene_data.index[:10])

# Save the gene data to the specified output path
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")



Preview of gene mapping DataFrame:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Shape of mapping dataframe: (45782, 2)



After mapping to gene symbols:
Gene data dimensions: 21278 genes × 69 samples
First 10 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE226244.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 19845 genes × 69 samples


Normalized gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE226244.csv
Extracting clinical features from the original source...
Extracted clinical features preview:
{'GSM7068800': [1.0], 'GSM7068801': [1.0], 'GSM7068802': [1.0], 'GSM7068803': [1.0], 'GSM7068804': [1.0], 'GSM7068805': [1.0], 'GSM7068806': [1.0], 'GSM7068807': [1.0], 'GSM7068808': [1.0], 'GSM7068809': [1.0], 'GSM7068810': [1.0], 'GSM7068811': [1.0], 'GSM7068812': [1.0], 'GSM7068813': [1.0], 'GSM7068814': [1.0], 'GSM7068815': [1.0], 'GSM7068816': [1.0], 'GSM7068817': [1.0], 'GSM7068818': [1.0], 'GSM7068819': [1.0], 'GSM7068820': [1.0], 'GSM7068821': [1.0], 'GSM7068822': [1.0], 'GSM7068823': [1.0], 'GSM7068824': [1.0], 'GSM7068825': [1.0], 'GSM7068826': [1.0], 'GSM7068827': [1.0], 'GSM7068828': [1.0], 'GSM7068829': [1.0], 'GSM7068830': [1.0], 'GSM7068831': [1.0], 'GSM7068832': [1.0], 'GSM7068833': [1.0], 'GSM7068834': [1.0], 'GSM7068835': [1.0], 'GSM7068836': [1.0], 'GSM7068837': [1.0], 'GSM7068

Data shape after handling missing values: (69, 19846)

Checking for bias in feature variables:
For the feature 'Psoriasis', the least common label is '0.0' with 8 occurrences. This represents 11.59% of the dataset.
The distribution of the feature 'Psoriasis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Psoriasis/GSE226244.csv
Final dataset shape: (69, 19846)
